# **NESTJS**

- [Cài đặt môi trường](#cài-đặt-môi-trường)  
- [Routing](#routing-get-post-put-delete)  
- [Middleware](#middleware)  
- [REQ/RES](#request--response)  
- [Tương tác DB](#tương-tác-db-database-migration)  
- [Kiến trúc mã nguồn](#kiến-trúc-mã-nguồn)  
- [Lifecycle](#lifecycle)  
- [Authen/Author](#authen-author-jwt)  
- [Exeption handle](#exception-handle)  

**NESTJS**: Framework xây dựng ứng dụng phía máy chủ dựa trên nodejs, hỗ trợ toàn diện với typescript. Mục tiêu cung cấp kiến trúc modular, dễ quản lý, bảo trì, thử nghiệm và mở rộng.  

### **CÀI ĐẶT MÔI TRƯỜNG**
*Cài đặt môi trương nestjs với cli*
```cmd
$ npm i -g @nestjs/cli
$ nest new project-name
$ npm run start
```
*Ứng dụng mặc định triển khai chạy trên cổng 3000*  
**MÔ HÌNH**  
- NestJS dựa trên expressjs, nên mọi kỹ thuật áp dứng mô hình MVC đều phù hợp với nest.
    - **M** *Model* : Dữ liệu, logic tổ chức dữ liệu nghiệp vụ: `Entity`, `Schema`, `Service`,... 
    - **V** *View*: Giao diện người dùng (Không trực tiếp trong nest - do REST API trả về json)
    - **C** *Controler*: Xử lý logic, trả về kết quả : `@Controler()` 
- Tổ chức mô hình trong nest: 
    - `controller/`: Định tuyến request
    -  Service: Logic xử lý 
    -  Entity/ Model: Thao tác lên DB, mỗi model có: Entity + Service + DTO, cung cấp khả năng mở rộng linh hoạt.

    *Minh hoạ tổ chức file 1 module - user*  
    src/  
    ┣ users/  
    ┃ ┣ dto/  
    ┃ ┃  ┣ create-user.dto.ts     
    ┃ ┃  ┣ delete-user.dto.ts    
    ┃ ┃  ┗ ...   
    ┃ ┣ users.module.ts  
    ┃ ┣ users.controller.ts  
    ┃ ┣ users.service.ts  
    ┃ ┗ user.schema.ts  
    ┗ ....


**CÁC THÀNH PHẦN**  
- **Routing**: (Định tuyến) - Cơ chế ánh xạ các URL lên các hàm xử lý phía máy chủ. KHi trình duyệt gửi req thông qua url, Nestjs định tuyến request đố đến 1 controler tương ứng với url nhận được.
- **Middleware**: (Trung gian) - Xử lý trước khi đưa req vào controller.  
- **Request/Response**: Các object yêu cầu/trả về giao tiếp giữa client và máy chủ  
- **Provider & Dependency Injection**: 
    - **Provider**: Các class đều có khả ăng inject như service  
    - **DI**: Các Constructor tự động đượ inject thông qua metadata
- **DTO** (Data Transfer Object): Định nghĩa cấu trúc dữ liệu **gửi vào** trong params để thực hiện.  
- **Guard**: Quyết định có cho phép request tiếp tục thực hiện hay không.  

**Quy trình khởi tạo API**  
```ts
// src/main.ts
// Tạo instance của ứng dụng nestjs dựa trên appmodule (module gốc)
const app = await NestFactory.create(AppModule);
await app.listen(PORT);
// app.module.ts
// Đăng ký: controler, provider
@Module({
  imports: [],
  controllers: [AppController],
  providers: [AppService],
})
export class AppModule {}
// app.controller.ts 
@Controller() // không có tham số - Controller gốc
export class AppController {
  constructor(private readonly appService: AppService) {}

  @Get()
  getHello(): string {
    return this.appService.getHello();
  }
}

```  
Khi api được gọi `GET http://localhost:3000/`,  
AppControler - định nghĩa trống `@Controller()` tìm kiếm ánh xạ đến `@Get`,  
Controller gọi tiếp đến `getHello()` trong appService

### **ROUTING (GET, POST, PUT, DELETE)**  
*Bảng annotation*  
| Annotation  | HTTP Method | Ý nghĩa                           | Ví dụ endpoint                    |
| ----------- | ----------- | --------------------------------- | --------------------------------- |
| `@Get()`    | GET         | Lấy dữ liệu                       | `@Get('users')` → `/users`        |
| `@Post()`   | POST        | Tạo mới dữ liệu                   | `@Post('create')` → `/create`     |
| `@Put()`    | PUT         | Cập nhật toàn bộ tài nguyên       | `@Put('user/:id')` → `/user/1`    |
| `@Patch()`  | PATCH       | Cập nhật một phần tài nguyên      | `@Patch('user')` → `/user`        |
| `@Delete()` | DELETE      | Xoá tài nguyên                    | `@Delete('user/:id')` → `/user/1` |
| `@All()`    | Tất cả      | Chấp nhận mọi method (GET, POST…) | `@All('ping')` → `/ping`          |

Kết hợp với controller để tạo thành endpoint hoàn chính: "user/create"  
*Decorator lấy dữ liệu từ request*  
| Decorator      | Lấy dữ liệu từ…                     | Cách sử dụng                           |
| -------------- | ----------------------------------- | -------------------------------------- |
| `@Body()`      | Body của request (POST, PUT, PATCH) | `create(@Body() dto: CreateUserDto)`   |
| `@Param()`     | URL params (`:id`, `:name`,...)     | `get(@Param('id') id: string)`         |
| `@Query()`     | Query string (`?name=value`)        | `get(@Query('name') name: string)`     |
| `@Headers()`   | Header trong request                | `get(@Headers('authorization') token)` |
| `@Ip()`        | Lấy địa chỉ IP của client           | `log(@Ip() clientIp)`                  |
| `@HostParam()` | Lấy tham số host trong subdomain    | `get(@HostParam('account') account)`   | 

*Minh hoạ* 
```ts
@Controller('user')
export class UserController {
    constructor(private userService: UserService) { }

    // ... /user
    @Post()
    create(@Body() createUserDto: CreateUserDto) {
        const res: UserSchema | "error" = this.userService.createNewUser(createUserDto);
        return res
    }

    // ... /user/get-by-name?name=
    @Get('get-by-name')
    getUserByName(@Query('name') name: string): UserSchema {
        return this.userService.getUserByName(name);
    }
}
```


### **MIDDLEWARE**

### **REQUEST / RESPONSE**

### **TƯƠNG TÁC DB, DATABASE MIGRATION**

### **KIẾN TRÚC MÃ NGUỒN**

### **LIFECYCLE**

### **AUTHEN/ AUTHOR (JWT)**

### **EXCEPTION HANDLE**